In [ ]:
import numpy
from isicarchive import jitfunc
from isicarchive.api import IsicApi
from isicarchive.font import Font
from isicarchive.sampler import Sampler, _sample_grid_2d
from matplotlib import pyplot
%matplotlib inline

f = Font(None)
s = Sampler()
k = s._kernels['lanczos3']

api = IsicApi()
image = api.image('ISIC_0000000')
image.load_image_data()
rimage = api.resample_image(image.data, (1200,1600))
api.set_text_in_image(rimage, 'This is merely a test...', fsize=0.04, min_alpha=1.0, color=[0,0,0], bcolor=[255,255,255])

pyplot.figure(figsize=(12,8))
pyplot.imshow(rimage)
pyplot.show()